# Tworzenie własnego modelu NER opartego o wyrażenia regularne i pattern matching

Instalacja pakietów spaCy

In [ ]:
!pip install spacy -U 
#!python -m spacy download pl_core_news_lg
#nie musimy pobierać modelu językowego dla polskiego, w tym przypadku tworzymy model "od zera"

Po instalacji należy zrestartować środowisko wykonawcze

In [1]:
import spacy
import regex as re
import pandas as pd
import numpy as np
from tqdm import tqdm



Załóżmy, że chcemy wykorzystać bibliotekę SpaCy do wykrywania oraz identyfikowanianumeru telefonu w tekście. 

Tekst: Nazywam się Jan Kowalski i mój numer telefonu to 123456789

Możemy do tego wykorzystać standardową bibliotekę regex oraz wyrażenia regularne:

In [2]:
tekst = "Nazywam się Jan Kowalski i mój numer telefonu to 123456789"

pattern = "\d{9}"

re.findall(pattern, tekst)

['123456789']

Tworząc zmienną pattern, do której przekazaliśmy nasze wyrażenie regularne oraz wykorzystując funkcję findall z pakietu regex, poprawnie wykryliśmy numer telefonu w tekście. Co jednak, jeśli chcemy do tego wykorzystać bibliotekę SpaCy oraz idące za nią dodatkowe informacje (m.in cechy morfosyntaktyczne) o tekście? W tym celu wykorzystamy mechanizm nazywany przez twórców biblioteki jako **Entity Ruler**

In [3]:
#Entity Ruler budujemy na PUSTYM MODELU językowym. Aby go wczytać, wykorzystujemy następującą formułę:

nlp = spacy.blank("pl")

#ISTOTNA INFORMACJA - pusty model muszą przyporządkować Państwo do zmiennej o innej nazwie,
#niż zmienna przechowująca model dla języka polskiego.

#Metoda blank tworzy "pusty model", w tym przypadku dla języka polskiego (argument) i przypisuje go do zmiennej nlp

In [4]:
#Następnie tworzymy obiekt klasy Ruler, który pozwala na przystosowanie modelu do naszych potrzeb

ruler = nlp.add_pipe("entity_ruler")

#W tym celu, tworząc zmienną "ruler" (może być ona oczywiście dowolnej nazwy) dodajemy do naszego pustego modelu 
#element pipelinu (potoku przetwarzania danych) element o nazwie "enttity_ruler", czyli moduł odpowiedzialny 
#za wykrywanie jednostek nazewniczych

In [7]:
#Następnie, tworzymy zmienną o nazwie patterns, w której przechowywać będziemy wzorce Regex dla interesujących nas elementów tekstu. 


patterns = [
                {
                    "label": "NUMER_TELEFONU", "pattern": [{"TEXT": {"REGEX": pattern}}
                                                        ]
                }
            ]

#Struktura obiektu przechowującego jest następująca:
#1. Pierwszy poziom to lista (cała zmienna patterns jest klasy list)
#2. Do listy "patterns" przekazujemy obiekt (bądź obiekty) klasy dict (słowniki)
#3. Każdy słownik składa się z dwóch par klucz:wartość
#4. Pierwsza para zawiera informację o etykiecie (label) dla naszej jednostki nazewniczej - w tym przypadku jest to "label": "TELEFON"
#5. Druga para zawiera wzorzec, który spełniać będzie stworzony przez nas warunek - "pattern": 
#6. Druga para składa się z klucza "pattern" oraz wartości w postaci listy słowników - [{"TEXT": {"REGEX": pattern}}]                                                       
#7. Do tej wartości przekazujemy słownik z kluczem REGEX oraz wartością w postaci właściwego wyrażenia regularnego, w tym przypadku wartością jest nasz regex
# stworzony wcześniej, który wyciąga nam numer telefonu - pattern = "\d{9}"

#Zbudowałem dla Państwa zmienną patterns w ten sposób, aby w łatwy sposób można było ją modyfikować - wystarczy jedynie zmienić wyrażenie regularne
#w zmiennej pattern, aby cały "wzorzec" uległ zmianie.





In [9]:
#Następnie do naszego rulera za pomocą metody .add_patterns przekazujemy naszą zmienną patterns zawierającą nasze wzorce
ruler.add_patterns(patterns)

#W ten sposób przygotowaliśmy nasz model, pozostaje nam sprawdzić jego działanie (w taki sam sposób, jak każdy inny model SpaCy)

#UWAGA! Jeśli będą Państwo eksperymentować z różnymi wzorcami, lepiej za każdym razem 
#uruchamiać cały kod nowa (od momentu stworzenia zmiennej nlp z pustym modelem dla języka polskiego).
#Różne stworzone przez Państwa wzorce mogą się na siebie nakładać, co powodować może błędy!!!!

#Dokumentacja z wykorzystaniem modułu znajduje się tutaj: https://spacy.io/usage/rule-based-matching

In [10]:
doc = nlp("Nazywam się Jan Kowalski i mój numer telefonu to 123456789")

In [12]:
for ent in doc.ents:
  print(ent.text, ent.label_)

123456789 PHONE_NUMBER
